In [6]:
import csv
import json
import sys

import requests
import pandas as pd
from bs4 import BeautifulSoup
from mlxtend.preprocessing import TransactionEncoder
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import ast

import os
import pdfkit
import pandas as pd
import hashlib
import numpy as np
import logging
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException


In [2]:
base_url_jobs = "https://www1.communitech.ca/api/search/jobs?networkId=628&hitsPerPage=20&page="

## Extraction

In [3]:
# Total number of pages (300 is estimate)
total_pages = 300

jobs_communitech = []

for page_number in range(0, total_pages + 1):  
    api_url = f"{base_url_jobs}{page_number}&filters=&query="
    response = requests.get(api_url)
    data = response.json()

    results = data["results"]
    for result in results:
        hits = result["hits"]
        for job in hits:
            job_details = {
                # "created_at": job.get("created_at", ""),
                "locations": job.get("locations", ""),
                # "organization_id": job["organization"]["id"],
                "organization_name": job["organization"]["name"],
                # "organization_logo_url": job["organization"]["logo_url"],
                "organization_slug": job["organization"]["slug"],
                "organization_topics": job["organization"]["topics"],
                "organization_industry_tags": job["organization"]["industry_tags"],
                # "organization_stage": job["organization"]["stage"],
                # "organization_head_count": job["organization"]["head_count"],
                "source": job.get("source", ""),
                "slug": job.get("slug", ""),
                "title": job.get("title", ""),
                "url": job.get("url", ""),
                "featured": job.get("featured", ""),
                "has_description": job.get("has_description", "")
            }
            
            jobs_communitech.append(job_details)

## Pre-processing

In [4]:
jobs_communitech = pd.DataFrame(jobs_communitech,columns=['locations','organization_name','organization_slug','organization_topics','organization_industry_tags','source','slug','title','url','featured','has_description'])
jobs_communitech['desc_url'] = 'https://www1.communitech.ca/companies/'+jobs_communitech['organization_slug']+'/jobs/'+jobs_communitech['slug']+'#content'
jobs_communitech['locations'] = jobs_communitech['locations'].apply(lambda x: x[0] if len(x) > 0 else x)
jobs_communitech.head(5)

,locations,organization_name,organization_slug,organization_topics,organization_industry_tags,source,slug,title,url,featured,has_description,desc_url
0,"Vancouver, BC, Canada",Semios,semios,"[Agriculture, B2B, C100 alumni, Data & analyti...","[Agriculture and Farming, Data and Analytics]",career_page,33826346-global-pr-and-communications-manager,Global PR and Communications Manager,https://apply.workable.com/semios/j/2FBA4A8612,False,False,https://www1.communitech.ca/companies/semios/j...
1,"Baltimore, MD, USA",Perpetua,perpetua-2,"[Advertising/marketing, B2B, Software]","[Advertising, Commerce and Shopping, Data and ...",career_page,33825592-senior-accountant-corporate-consolida...,"Senior Accountant, Corporate Consolidation",https://jobs.lever.co/FlywheelDigital/56141099...,False,True,https://www1.communitech.ca/companies/perpetua...
2,"Baltimore, MD, USA",Perpetua,perpetua-2,"[Advertising/marketing, B2B, Software]","[Advertising, Commerce and Shopping, Data and ...",career_page,33825590-fp-a-revenue-manager,FP&A Revenue Manager,https://jobs.lever.co/FlywheelDigital/155817b7...,False,True,https://www1.communitech.ca/companies/perpetua...
3,"Toronto, ON, Canada",Perpetua,perpetua-2,"[Advertising/marketing, B2B, Software]","[Advertising, Commerce and Shopping, Data and ...",career_page,33825589-advertising-account-manager,Advertising Account Manager,https://jobs.lever.co/FlywheelDigital/0522d215...,False,True,https://www1.communitech.ca/companies/perpetua...
4,"Toronto, ON, Canada",Perpetua,perpetua-2,"[Advertising/marketing, B2B, Software]","[Advertising, Commerce and Shopping, Data and ...",career_page,33825587-account-executive-emerging,"Account Executive, Emerging",https://jobs.lever.co/FlywheelDigital/101219ad...,False,True,https://www1.communitech.ca/companies/perpetua...


In [5]:
if 'description' not in jobs_communitech.columns:
    jobs_communitech['description'] = pd.NA


for index, row in jobs_communitech.iterrows():
    if row['has_description']:
        response = requests.get(row['desc_url'])
        soup = BeautifulSoup(response.text, 'html.parser')
        jobs_communitech.at[index, 'description'] = soup.get_text()

jobs_communitech.drop(columns=['desc_url','has_description','source','slug','organization_slug'], inplace=True)

jobs_communitech = jobs_communitech[['title','locations','organization_name','organization_topics','organization_industry_tags','description','url','featured']]
jobs_communitech.columns = ['Title','Location','Company','Topics','Tags','Description','URL','Featured']
jobs_communitech.head(3)


,Title,Location,Company,Topics,Tags,Description,URL,Featured
0,Global PR and Communications Manager,"Vancouver, BC, Canada",Semios,"[Agriculture, B2B, C100 alumni, Data & analyti...","[Agriculture and Farming, Data and Analytics]",<NA>,https://apply.workable.com/semios/j/2FBA4A8612,False
1,"Senior Accountant, Corporate Consolidation","Baltimore, MD, USA",Perpetua,"[Advertising/marketing, B2B, Software]","[Advertising, Commerce and Shopping, Data and ...","Senior Accountant, Corporate Consolidation @ P...",https://jobs.lever.co/FlywheelDigital/56141099...,False
2,FP&A Revenue Manager,"Baltimore, MD, USA",Perpetua,"[Advertising/marketing, B2B, Software]","[Advertising, Commerce and Shopping, Data and ...",FP&A Revenue Manager @ Perpetua | Work In Tech...,https://jobs.lever.co/FlywheelDigital/155817b7...,False


In [6]:
jobs_communitech.describe(include='all').T

,count,unique,top,freq
Title,5661,3982,Sales Development Representative,51
Location,5661,376,"Toronto, ON, Canada",901
Company,5661,763,Dataperformers,217
Topics,5661,639,"[AI/ML, B2B, Media & entertainment, SaaS, Soft...",217
Tags,5661,564,"[Biotechnology, Data and Analytics, DeepTech, ...",217
Description,3787,3772,"VP, Sales Operations @ Benevity | Work In Tech...",3
URL,5661,5616,https://jobs.lever.co/axiomzen/ef303c33-3df5-4...,2
Featured,5661,1,False,5661


In [7]:
url_companies = "https://jobs.golden.ventures/api-boards/search-companies"

payload = json.dumps({
  "query": {
    "parent": "golden-ventures",
    "promoteFeatured": True
  },
  "meta": {
    "size": 1000
  },
  "board": {
    "id": "golden-ventures",
    "isParent": True
  }
})
headers = {
  'authority': 'jobs.golden.ventures',
  'accept': 'application/json',
  'accept-language': 'en-US,en;q=0.9',
  'cache-control': 'no-cache',
  'content-type': 'application/json',
  'cookie': 'session=eyJmbGFzaCI6e319; session.sig=K1cJpOd8V9qlNgDIOLT2owA9zqw',
  'dnt': '1',
  'origin': 'https://jobs.golden.ventures',
  'pragma': 'no-cache',
  'referer': 'https://jobs.golden.ventures/companies',
  'sec-ch-ua': '"Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'same-origin',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
}

response = requests.request("POST", url_companies, headers=headers, data=payload)
companies_g = json.loads(response.text)
companies_golden = pd.DataFrame(companies_g['companies'])

companies_golden.head(3)

,description,domain,emailDomains,id,investorSlugs,investors,logos,markets,marketsLV,officeLocations,...,parentSlugs,parents,slug,stages,stagesLV,fundingLV,sizeLV,isFeatured,isRemoteFriendly,website
0,"At ApplyBoard, we believe that education is a ...",applyboard.com,[applyboard.com],ApplyBoard,[golden-ventures],[Golden Ventures],"{'manual': {'height': 160, 'src': 'https://dzh...",[Education Management],"[{'id': 'Education Management', 'label': 'Educ...","[Kitchener, Ontario, Canada]",...,"[anthos-capital, golden-ventures]","[Anthos Capital, Golden Ventures]",applyboard,"[1000+ employees, Growth]","[{'id': '1000+ employees', 'label': '1000+ emp...","{'id': 'Growth', 'label': 'Growth', 'value': '...","{'id': '1000+ employees', 'label': '1000+ empl...",False,False,"{'url': 'https://applyboard.com/', 'label': 'a..."
1,Neo Financial is a leading Canadian financial ...,neofinancial.com,[neofinancial.com],Neo Financial,"[gaingels, golden-ventures, thomvest]","[Gaingels, Golden Ventures, Thomvest]","{'manual': {'height': 160, 'src': 'https://dzh...","[Financial Services, Fintech]","[{'id': 'Financial Services', 'label': 'Financ...","[Calgary, Alberta, Canada, Winnipeg, Manitoba,...",...,"[gaingels, golden-ventures, thomvest]","[Gaingels, Golden Ventures, Thomvest]",neo-financial,"[100-1000 employees, Growth]","[{'id': '100-1000 employees', 'label': '100-10...","{'id': 'Growth', 'label': 'Growth', 'value': '...","{'id': '100-1000 employees', 'label': '100-100...",False,False,"{'url': 'https://neofinancial.com/', 'label': ..."
2,Faire is an online wholesale marketplace built...,faire.com,[faire.com],Faire,"[forerunner-ventures, golden-ventures, lightsp...","[Forerunner Ventures, Golden Ventures, Lightsp...","{'manual': {'height': 160, 'src': 'https://dzh...",[Retail],"[{'id': 'Retail', 'label': 'Retail', 'value': ...","[Greater Los Angeles Area, Salt Lake City, Uta...",...,"[forerunner-ventures, golden-ventures, lightsp...","[Forerunner Ventures, Golden Ventures, Lightsp...",faire,[1000+ employees],"[{'id': '1000+ employees', 'label': '1000+ emp...",NaN,"{'id': '1000+ employees', 'label': '1000+ empl...",False,False,"{'url': 'https://faire.com/', 'label': 'faire...."


In [8]:
companies = companies_golden.slug.unique()

In [9]:
url_jobs_golden = "https://jobs.golden.ventures/api-boards/search-jobs"

all_data_jobs_golden = []  

for company in companies:
  payload = json.dumps({
      "meta": {
          "size": 400
      },
      "board": {
          "id": company,  
          "isParent": False
      },
      "query": {
          "remoteOnly": False,
          "internshipOnly": False,
          "managerOnly": False,
          "bestFit": False
      },
      "parentSlug": company
  })
  headers = {
    'authority': 'jobs.golden.ventures',
    'accept': 'application/json',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'cookie': 'session=eyJmbGFzaCI6e319; session.sig=K1cJpOd8V9qlNgDIOLT2owA9zqw',
    'dnt': '1',
    'origin': 'https://jobs.golden.ventures',
    'pragma': 'no-cache',
    'referer': 'https://jobs.golden.ventures/jobs/bluedot-global',
    'sec-ch-ua': '"Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'
  }
  response = requests.request("POST", url_jobs_golden, headers=headers, data=payload)
  data = json.loads(response.text)
  if 'jobs' in data:
      jobs_data_2 = data['jobs']
      df = pd.DataFrame(jobs_data_2)
      all_data_jobs_golden.append(df)
  else:
      print(f"The key 'jobs' does not exist in the response data for company: {company}.")

# Concatenate all dataframes
jobs_golden = pd.concat(all_data_jobs_golden, ignore_index=True)
jobs_golden.head()

,applyUrl,companyDomain,companyLogos,companyId,companySlug,companyName,considerHosted,departments,jobTypes,jobFunctions,...,title,url,jobId,remote,scores,atsJobs,matchingTalent,jobSeniorityIds,isFeatured,maxYearsExp
0,https://applyboard.wd3.myworkdayjobs.com/Apply...,applyboard.com,"{'manual': {'height': 160, 'src': 'https://dzh...",ApplyBoard,applyboard,ApplyBoard,False,[],"[{'id': 'manager-recruiter', 'label': 'Recruit...","[{'id': 'Human Resources', 'label': 'Human Res...",...,International Recruitment Manager,https://applyboard.wd3.myworkdayjobs.com/Apply...,117d19f080d71001d8b08ba01a360000,False,"{'score': 1001.0349097222222, 'matchScore': 0,...",[],"{'count': 0, 'matches': []}",[manager],False,NaN
1,https://applyboard.wd3.myworkdayjobs.com/Apply...,applyboard.com,"{'manual': {'height': 160, 'src': 'https://dzh...",ApplyBoard,applyboard,ApplyBoard,False,[],"[{'id': 'manager-recruiter', 'label': 'Recruit...","[{'id': 'Human Resources', 'label': 'Human Res...",...,International Recruitment Manager,https://applyboard.wd3.myworkdayjobs.com/Apply...,117d19f080d71001d7de945fff390000,False,"{'score': 1.0349097222222223, 'matchScore': 0,...",[],"{'count': 0, 'matches': []}",[manager],False,NaN
2,https://applyboard.wd3.myworkdayjobs.com/Apply...,applyboard.com,"{'manual': {'height': 160, 'src': 'https://dzh...",ApplyBoard,applyboard,ApplyBoard,False,[],"[{'id': 'sales-manager', 'label': 'Sales Manag...","[{'id': 'Sales', 'label': 'Sales', 'value': 'S...",...,Senior Sales Manager,https://applyboard.wd3.myworkdayjobs.com/Apply...,117d19f080d71001a0397156c0c40000,False,"{'score': 1.001576388888889, 'matchScore': 0, ...",[],"{'count': 0, 'matches': []}",[manager],False,NaN
3,https://applyboard.wd3.myworkdayjobs.com/Apply...,applyboard.com,"{'manual': {'height': 160, 'src': 'https://dzh...",ApplyBoard,applyboard,ApplyBoard,False,[],"[{'id': 'manager-recruiter', 'label': 'Recruit...","[{'id': 'Human Resources', 'label': 'Human Res...",...,International Recruitment Officer,https://applyboard.wd3.myworkdayjobs.com/Apply...,117d19f080d71001393a3885a4ae0000,False,"{'score': 0.9941091820987654, 'matchScore': 0,...",[],"{'count': 0, 'matches': []}",[],False,NaN
4,https://applyboard.wd3.myworkdayjobs.com/Apply...,applyboard.com,"{'manual': {'height': 160, 'src': 'https://dzh...",ApplyBoard,applyboard,ApplyBoard,False,[],"[{'id': 'manager-recruiter', 'label': 'Recruit...","[{'id': 'Human Resources', 'label': 'Human Res...",...,International Recruitment Manager,https://applyboard.wd3.myworkdayjobs.com/Apply...,a94f64776b7910013f864134b77b0000,False,"{'score': 0.9682430555555556, 'matchScore': 0,...",[],"{'count': 0, 'matches': []}",[manager],False,NaN


## Pre-processing

In [10]:
jobs_golden.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
applyUrl,391,391,https://applyboard.wd3.myworkdayjobs.com/Apply...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
companyDomain,391,37,applyboard.com,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
companyLogos,391,37,"{'manual': {'height': 160, 'src': 'https://dzh...",94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
companyId,391,37,ApplyBoard,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
companySlug,391,37,applyboard,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
companyName,391,37,ApplyBoard,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
considerHosted,391,1,False,391,NaN,NaN,NaN,NaN,NaN,NaN,NaN
departments,391,33,[],258,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jobTypes,391,154,"[{'id': 'manager-recruiter', 'label': 'Recruit...",34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jobFunctions,391,35,"[{'id': 'Engineering', 'label': 'Engineering',...",90,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
def extract_labels(label_list):
    if isinstance(label_list, list):
        return [label['label'] for label in label_list]
    return []

jobs_golden['jobTypes'] = jobs_golden['jobTypes'].apply(extract_labels)
jobs_golden['jobFunctions'] = jobs_golden['jobFunctions'].apply(extract_labels)
jobs_golden['normalizedLocations'] = jobs_golden['normalizedLocations'].apply(extract_labels)
jobs_golden['skills'] = jobs_golden['skills'].apply(extract_labels)
jobs_golden['requiredSkills'] = jobs_golden['requiredSkills'].apply(extract_labels)
jobs_golden['regions'] = jobs_golden['regions'].apply(extract_labels)
jobs_golden['stages'] = jobs_golden['stages'].apply(extract_labels)
jobs_golden['markets'] = jobs_golden['markets'].apply(extract_labels)
jobs_golden['preferredSkills'] = jobs_golden['preferredSkills'].apply(extract_labels)

In [12]:
jobs_golden.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
applyUrl,391,391,https://applyboard.wd3.myworkdayjobs.com/Apply...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
companyDomain,391,37,applyboard.com,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
companyLogos,391,37,"{'manual': {'height': 160, 'src': 'https://dzh...",94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
companyId,391,37,ApplyBoard,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
companySlug,391,37,applyboard,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
companyName,391,37,ApplyBoard,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
considerHosted,391,1,False,391,NaN,NaN,NaN,NaN,NaN,NaN,NaN
departments,391,33,[],258,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jobTypes,391,154,[Recruiting Manager],34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jobFunctions,391,35,[Engineering],90,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Create a list of titles
titles =jobs_communitech['Title'].unique().tolist()
titles_2 = jobs_golden['title'].unique().tolist()
titles = titles + titles_2
titles

['Global PR and Communications Manager',
 'Senior Accountant, Corporate Consolidation',
 'FP&A Revenue Manager',
 'Advertising Account Manager',
 'Account Executive, Emerging',
 'Regional Sales Manager, Home Depot - Western Canada',
 'Collections Specialist',
 'Sr Manager, SaaS Procurement and Procurement Operations',
 'Software Engineer',
 'Full Stack Engineer, Wavelo',
 'Senior Business Systems Analyst',
 'Technical Lead Test Engineer - MDM',
 'Director, Reporting',
 'Accounting Associate',
 'Customer Care Professional',
 'Partenaire en acquisition de talent',
 'Director, Talent & Organizational Development',
 'Assistant Store Manager (Rideau- Ottawa)',
 'Senior Software Developer',
 'Staff Software Engineer',
 'Senior Full Stack Developer',
 'Channel Marketing Manager - EMEA',
 'Software Engineering Manager',
 'Senior Firmware Engineer',
 'Ingénieur.e senior firmware',
 'Technical Writer',
 'Global Inventory and MRP Lead',
 'Senior Application Engineer',
 'Application Engineer EMEA'

In [13]:
titles = [title.split(' - ')[0] for title in titles]
titles = [title.split(' (')[0] for title in titles]
titles = [title.split(' | ')[0] for title in titles]
titles = [title.split(', ')[0] for title in titles]
titles = [title.split(' / ')[0] for title in titles]
titles = [title.lower() for title in titles]
titles = [title.replace('(canada) ', '').replace('(us) ', '').replace('(senior) ', '').replace('(マーケティングマネージャー)','').strip() for title in titles]
titles = list(set(titles))  
titles.sort()
titles

['(us)solutions analyst',
 '(usa) growth insights manager',
 '- software engineer position',
 '.net developer',
 '.net software developer',
 '2023 aquamobile swim instructor and/or lifeguard',
 '2024',
 '209403-terra phoenix engineering',
 '309a electrician',
 '442a electrician',
 '[contract] go-to-market talent sourcer',
 '[position filled] data scientist',
 'aba clinic software implementation specialist',
 'abm marketing manager',
 'accessibility tester',
 'account based marketing manager',
 'account development executive',
 'account development manager',
 'account development representative',
 'account director',
 'account executive',
 'account executive -canada and usa',
 'account management team lead',
 'account manager',
 'account manager inside sales',
 'account manager/senior manager',
 'account representative',
 'account specialist',
 'account support officer',
 'account support specialist',
 'accountant',
 'accounting',
 'accounting administrator',
 'accounting analyst',
 'ac

In [ ]:
len(titles)

In [ ]:
# Set the URL and headers
payload = {}
headers = {
  'authority': 'www.livecareer.com',
  'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
  'accept-language': 'en-US,en;q=0.9',
  'cache-control': 'no-cache',
  'cookie': 'x-georegion=295,CA,ON,TORONTO,,,,,,43.67,-79.42,EST,M3H+M3M+M4B+M4C+M4E+M4G+M4H+M4J+M4K+M4L+M4M+M4N+M4P+M4R+M4S+M4T+M4V+M4W+M4X+M4Y+M5A+M5B+M5C+M5E+M5G+M5H+M5J+M5K+M5L+M5M+M5N+M5P+M5R+M5S+M5T+M5V+M5W+M5X+M6A+M6B+M6C+M6E+M6G+M6H+M6J+M6K+M6L+M6M+M6N+M6P+M6R+M6S+M7A+M7Y+M9M+M9N+M9P+M9W,NA,,,802,vhigh,5000; vssessionuid=edc7e3c7-125e-4517-bc17-1e87a377baa7; vstr=44e7a059-9df4-40eb-a9bc-a214f1ba0e51; vsuid=83923cf0-d086-4342-ae22-ad7c2ef0b072; ref=14; visitinfo=[City,North York]&[State,ON]&[Country,CA]&[PostalCode,M3J]&[BrowserName,Chrome]&[BrowserVersion,120]&[DeviceType,]&[OSName,Windows]&[DeviceModel,Unknown]&[OSVersion,10.0]; vsutms=8846de03-0f2b-4b70-870c-5771d4b61295#44e7a059-9df4-40eb-a9bc-a214f1ba0e51#83923cf0-d086-4342-ae22-ad7c2ef0b072#1703736192#https://google.com#||||; vsintentcd=RSGNRL; ci_id=dc617d47-a16f-442b-afa5-t18cae972f2d-e76f342896dc; ARRAffinity=04e4ae4662bb6d686af058e5e5dc2f336bdaef569f9a9b7df71508476ed2041f; ARRAffinitySameSite=04e4ae4662bb6d686af058e5e5dc2f336bdaef569f9a9b7df71508476ed2041f; Auth=tZD-nTtxJSfHBh-UPP6UEX5aGGD7RT7ePjtoI616JXRZKcfA6NYzXpySLcHL86E1f7-xi8oqZI3iiiUetJkCbHhKP4qD3pfsAVmyRZGtaEEnX2OAGWtENeOeef4Vi3yXfH8IzRCgqUQL1NUnf-nbmifEQN2wA087ivT37M-nJ5WPhFIOq593fOhVJ-uCbmPMKTvKVhHUpewTgzv61TyPMOtnxNMnIpLeYrYogpSWIiXcGWcWp_n8U3t6XZ-GMqF1SWc-VwK2H6rfJuBUwRnM1izratGTxaP7qFTIuc4RzF2ZhjsAs2vxOLxorEEwv1RnvAVEgE0j3rP4K35YOYYE0aKakyNPHMnE7YON4tgZC3cRS6kHID6L2Ag__aiYGDXx4sX5mFzDZelkQDZ0VAppdK1sqgh2GtoPX0HhYEB0UD3mgzcuktG6NkZIwZ6k-ucIz2nRniqU_UqrPRuXl4YjcLxNtBSR6UitAhbzWK8cMBQpuHsUSAQaaptw8SDorKnbQd-UFdC1tA04RdfzApok_q7urdLwICWUSWNk6fvy09HFuJJorlKipsCJWCLBrMXVud6BwgxcmRzX0DB6N9TMlCHmM0i1puNmzGD_FqPq0Sc6nffmPWTw9_ei7vG7HEjQF8jPMdQa5xKOkHWmEfwDqmg5P84iBIpRgFm18Ns5QWhEw5hIBj0w4vtEQjrfCz5HDMVeHxbnvXuHsJF2dlsN_IvfRlQI85P0jwpjEbyoXZICmAvVeBHrT7p1WrtxQFoOu8xesRgnXZ3RCvvGsteq0D0j_ltMJhSB8qrQVgE2ZTg1G7ed_iyZDku1UdoSbU-qJYut7ZA7IXZ0MgvI30aBFAbWPQ61bAzx993ax6s65vHyklRP06IUEN9rXPaP9S6wlA9lcSG18tZfHYRv9_k3I_08DBmfwsUzkwK4igN_TTZxk7cNvsewUNwVWMbCUI8WfU8Ni7Y911XiwmGzj1Y9qya6rYcD_leyI-RRiTmh1-4v_o9URlT1z66Y0ShhippQ; UserStatus={"IsUserLoggedIn":false,"User":{"Role":0,"UserId":"847d4a5f-9a42-48c9-8eac-450c86379d15","AccDisplayName":"Guest","CreatedOn":"12/28/2023 5:30:28 AM"}}; acc_session={ec77e065-8bed-4bde-9399-4997c66294a9}; isFirstTimeUser=0; DocumentID=fa1d4d7e-1452-90c3-e316-ed51b5b09287; bm_sz=A5496B0C2D3CBDE33F6425F7AEC79CCD~YAAQ5mUzuC5KM2eMAQAAhcMIsRaPm6BXjg5oIsXlVJeqwXdsbDy4bGV0eCuk2taG7rc9fSM1fzzXFSRiE297o4KA4H/ym/ALT6ZgkqN5X7z+gruIkmV4eJ41uZaIGT1Wh9K/PO0iVD3omBh7Ni2Lu1RAY6UMVWLBnav7gT8nDxQpK6jbDUdx+9gy1SMNb1sIiIxIituiC/FQAB191zlwz1XnxaSkb5sNz1Lsgb3wl/Euu2Qa+RRcNpAH4E48Mv/A8m0Cl+MphGLK/rflwH3yFOjT7ZvcCiCGZxoad4eFzlWZTQq+WU9w~3420997~3551554; _abck=3220C70AB729592693CB0CF1DBD0D013~0~YAAQ5mUzuAFPM2eMAQAAvToJsQveF1VnQN0qe0HXz8EwMG7ECaadmMc+Z+DMvmB4P8iMehswJXpR0YHqn4NJfOq18zh/xWtNH4N1HS4WsOQuI8DUqdJhPqx9oN6L+h9vYT8p6BfiEecn680QH/AYuE+WjjV3qtjhq4ZFdWROo4wQNFYLGh2uRTvVNiOtK87N/kfQYw+OU1GiAFZog6q80AczDedYEsSMChkj8ju6bjDq8ex8s2Xb6GgW0BHnx2jxONMp1JY4wnT4FxeByE3ZJxM4WUMPMKO+dgFnNOxQkwFu9nC5NCUvrArLH9thY6Hsg+ssmJsEaOQE+wnl6nUQVUTahaY9AYF/IQhRW1lh3PAmMxNYwQRYZZ280uZGRTzv4JLNhCuUMG4ZQMoPCpwyre6v8ROBa9K055S6qg==~-1~-1~-1; bm_mi=7E3B1A39D0786EDDC1250549CCD493C4~YAAQ5mUzuLZTM2eMAQAAZqIJsRYeOVBlszn/AmZi7p5SavD3N3qOPMqu8u1JGviVIdF77HX8uyWHzcVMkdp48XnZrdwVOdpgcRbvZO2vxQYJftHYchvHfQFSR/qp2QIJEZzm18Us9W7BO8yIlYTpLH8xDLw20KfTN66P4bhKupRcOCdmDghqk69TQyN5LeRFTuUGt/a9S7YxzCZWMDko3A/4ywXgcfOJA++31EpFhxW17LK9frRQmroorMvWSvrj1+qlCpbfOYU3cAbRBohKY16cVvb2Vn/J2aPGchrMHyf0WtqElW6jgHVbwBsYGJ5Do/yC4sUBYfqanvtm~1; bm_sv=6F5884ED906C4E66C9D8DEAC55CF9ECB~YAAQ5mUzuEtUM2eMAQAANLQJsRb9JOVI7QlRAd1iKn2JbP2Gxr8LV/4RwA9jvpGD6AAizF7Hv2//R94mQbZr2AO1InFXkV2KgyBc6kKvGhrykUPcYWknumUeig1vf7t/0+Qv94pO1Kq5e4zWLC/H+bZv7B6ovpjvEWQvoLzAJhB2hwdTg5TnEXcuP24nXEbYFbMWrphrWkNV+4GUwWXgUtPgIBPof912KTKIt1/ku8f36BD0xCvnZtSETa9qWO7Ds1hzEUU=~1; ak_bmsc=F75787051AB05D2549E2D10A9E447D1F~000000000000000000000000000000~YAAQ5mUzuIlXM2eMAQAAbBAKsRYG0Jc6ZAVZkigfZkpC4RHX4NXg90EryYruj7aA4DUh227H6WEpsqXSzBY+v6xapJgz9wB2wMWAaj+j8GBiT+IRqm4Rqe7LWO918BRmCGGLcEd4G0DyxLqSaGRX0A7tgzkU6Y1uwpSr9obTaFIoQYKTCmt9rBv/0FKHMhH39ZmK6jkAZf5vzPhMAKrDiywdNxKGcvlZ04ODRsz8nJJfWbPWAMlfSxxDUu8cvw2yX+17wYZCqQ0RSFmxmar9jCmxvwCeSF4njAYdh40fSuKh6InfBE6Tldtnzj9rsELI7RMZzK8TYMha/5k9G6az1ikru5ZWXi3AObMYQglYATPEIAq6SCul+ce5UcZKuSJGOdBItH4B0peeiCWHN/4DHeR8uFglXEWPy1uHs2hHPTWpQYlLA6vOH2CFW4c=; TtfbData={"countryCD":"CA","expId":"","isCache":true,"reqTst":1703777881000,"resTst":1703777881000}; _abck=3220C70AB729592693CB0CF1DBD0D013~-1~YAAQ5mUzuHK0M2eMAQAAAoIUsQtGXtxj9d2AV6PC5f8yupKPSFIAjPklqVyd4CA4yaV5/bEszfXXMi/600kY5pB90LyhVIRAYbE8sRAeUYPFShvgcmkNUktT995LhRUSnoqg7FINqEd1MEUdSqYJwan+gblIUIALw4jNLD6MNQWxi69e3/LGURA+ZXmJbA4mIvi2QbyUs1/JP/z+8Um1YbaDSmNl3geKxZP/lRxJQ86ng17jK1qhvgjkookpR/bN7VKjqwXbufTvg85JB2wPL0LxQJH11jIym769AdSwLDpqmj75kcFPR0azEyNGE8p0Y3M5QJBjCApGg9M1Yko9VbyBB43gmo3OryASJErfDqT9aQxoZAi7zq+e1CHdmupBVLEeiRFwxnlQ2tJHPlriiRuevTnBTlVqRC63Yw==~0~-1~-1',
  'dnt': '1',
  'pragma': 'no-cache',
  'referer': 'https://www.livecareer.com/resume-search',
  'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'document',
  'sec-fetch-mode': 'navigate',
  'sec-fetch-site': 'same-origin',
  'sec-fetch-user': '?1',
  'upgrade-insecure-requests': '1',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

In [ ]:
# set up logging
logging.basicConfig(level=logging.INFO)

In [ ]:
# creating an empty list to store the data
total_data = []

In [ ]:
for job in titles:
    JOB = job.lower()
    link_counter = 0  
    PAGE = '1' 

    URL = "https://www.livecareer.com/resume-search/search?jt=" + JOB + "&bg=85&eg=100&comp=&mod=&pg=" + PAGE
    response = requests.get(URL, headers=headers, data=payload)

    if response.status_code != 200:
        logging.warning(f"Failed to fetch data for {URL}")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    resume_items = soup.find_all('a', class_='sc-1dzblrg-0 caJIKu sc-1os65za-2 jhoVRR')

    for item in resume_items:
        if link_counter >= 4:
            break 

        title = item.find('p', class_='title').get_text(strip=True)
        company = item.find('p', class_='company-title').get_text(strip=True)
        location = item.find('p', class_='company location').get_text(strip=True)
        updated = item.find('p', class_='last-updated').get_text(strip=True)
        resume_url = item['href']

        total_data.append({
            'Title': title,
            'Company': company,
            'Location': location,
            'Last Updated': updated,
            'URL': f"https://www.livecareer.com{resume_url}"
        })

        link_counter += 1 
        
# convert to Dataframe
resume_data = pd.DataFrame(total_data)
resume_data.head()

In [ ]:
# total shape of the dataframe
resume_data.shape

In [ ]:
# drop duplicates
resume_data = resume_data.drop_duplicates()
resume_data = resume_data.reset_index(drop=True)
resume_data.shape

In [ ]:
# Add empty columns for Raw_html and Resume
resume_data['Raw_html'] = ""
resume_data['Resume'] = ""

In [ ]:
# Set the path to your Chrome webdriver executable 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("webdriver.chrome.driver=path/to/chromedriver.exe")

In [ ]:
# Initialize the Chrome webdriver with the specified options
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
for i in range(df.shape[0]):
    url = resume_data['URL'].iloc[i]  
    driver.get(url)
    
    try:
        # Wait for the specific element to be loaded. In this case, the element with ID "server-resume-template"
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#server-resume-template > div > div"))
        )
        
        # Get the page source and process it
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Find the relevant div containing the resume
        resume_div = soup.select_one("#server-resume-template > div > div")
        raw_html = str(resume_div) if resume_div else ""
        resume_text = resume_div.get_text(separator=" ", strip=True) if resume_div else ""

        # Update the DataFrame
        df.at[i, 'Raw_html'] = raw_html
        df.at[i, 'Resume'] = resume_text

    except TimeoutException:
        print(f"Timeout while loading {url}")
    except Exception as e:
        print(f"An error occurred while processing URL {url}: {e}")
        df.at[i, 'Raw_html'] = ""
        df.at[i, 'Resume'] = ""


In [ ]:
# Order columns
resume_data = resume_data[['Title','Raw_html', 'Resume']]
resume_data.head()

Writing to file :

In [13]:
# Write DataFrame to CSV file for jobs_communitech
jobs_communitech.to_csv('communitech_jobs.csv', index=False, encoding='utf-8')
# Write DataFrame to CSV file for jobs_golden
jobs_golden.to_csv('golden_jobs.csv', index=False, encoding='utf-8')
# Write DataFrame to CSV file for resume_data
resume_data.to_csv('resume_data.csv', index=False)